<a href="https://colab.research.google.com/github/Elshamysamira/Information-Extraction-and-Retrieval/blob/nasti/Information_Extraction_and_Retrieval_with_SQLite_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install nltk
!pip install chardet
import nltk
import chardet
nltk.download('punkt')  # This downloads necessary datasets for tokenization
from nltk.tokenize import word_tokenize
from collections import defaultdict
from pathlib import Path
import os
import sqlite3

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from pathlib import Path
import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
# Connect to SQLite database
conn = sqlite3.connect('/content/drive/My Drive/Documents/inverted_index.db')
cursor = conn.cursor()

# Create table for inverted index
cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                    word TEXT PRIMARY KEY,
                    document_ids TEXT
                )''')

def printing_of_file(files):
    for file in files:
        print(file)

folder_path = '/content/drive/My Drive/Documents'
files = os.listdir(folder_path)
printing_of_file(files)

A thousand ways to make money.txt
Dumbells of Business by Louis Custer Martin Reed.txt
Business Administration- Theory, Practice and Application.txt
Cyclopedia of Commerce, Accountancy, Business Administration, v. 01 (of 10).txt
Confessions of a Tradesman by Frank Thomas Bullen.txt
Fundamentals of Prosperity- What They Are and Whence They Come by Roger Ward Babson.txt
The Knack of Managing by Lewis K. Urquhart and Herbert Watson.txt
Cyclopedia of Commerce, Accountancy, Business Administration, v. 04 (of 10).txt
Creating Capital by Frederick L. Lipman.txt
Twenty Years of Hus'ling by J. P. Johnston.txt
Business Hints for Men and Women by A. R. Calhoun.txt
The Young Man in Business by Edward William Bok.txt
Forging Ahead in Business by Alexander Hamilton Institute.txt
Fifty years in Wall Street by Henry Clews.txt
Cyclopedia of Commerce, Accountancy, Business Administration.txt
Increasing Human Efficiency in Business by Walter Dill Scott.txt
Cyclopedia of Commerce, Accountancy, Business Ad

In [9]:
def tokenize_and_save_documents(documents, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Create the output directory if it doesn't exist

    tokenized_docs = {}
    for documentID, document_path in enumerate(documents):
        try:
            # Open and read the document content
            with open(document_path, 'r', encoding='utf-8', errors='ignore') as file:
                document_content = file.read()

            # Tokenize the document content
            tokens = word_tokenize(document_content)
            tokenized_docs[documentID] = tokens

            # Save the tokenized content to a file
            output_file_path = os.path.join(output_dir, f"tokenized_document_{documentID}.txt")
            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                output_file.write(' '.join(tokens))  # Write tokens separated by spaces

        except Exception as e:
            print(f"Error processing document {document_path}: {e}")

    return tokenized_docs


In [10]:
# Tokenize documents and save the results
tokenized_books = tokenize_and_save_documents(all_books, output_directory)
print("Tokenization complete and files saved.")

# Initialize the inverted index as a defaultdict of sets
inverted_index = defaultdict(set)

Tokenization complete and files saved.


In [12]:
for documentID, document_path in enumerate(all_books):
    try:
        with open(document_path, 'r', encoding='latin-1') as file:
            document_content = file.read()

        # Tokenize by splitting on whitespace and standardize to lowercase
        for word in document_content.lower().split():
            inverted_index[word].add(documentID)  # Use set to avoid duplicate document IDs

    except Exception as e:
        print(f"Error processing document {document_path}: {e}")

# Convert sets to lists for storing in SQLite database
for word in inverted_index:
    document_ids = ','.join(map(str, inverted_index[word]))  # Convert set to comma-separated string
    cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, document_ids))

# Commit changes and close connection
conn.commit()
conn.close()

print("Inverted index has been saved successfully in SQLite database.")


Inverted index has been saved successfully in SQLite database.


In [14]:
# Assuming all_books is a list of Path objects
document_mapping = {documentID: document.name for documentID, document in enumerate(all_books)}
def lookup_word(word):
    conn = sqlite3.connect('/content/drive/My Drive/Documents/inverted_index.db')
    cursor = conn.cursor()

    cursor.execute("SELECT document_ids FROM inverted_index WHERE word=?", (word,))
    result = cursor.fetchone()
    conn.close()

    if result:
        document_ids = result[0].split(',')
        print(f"Congratulations! I could find the word '{word}' in the following document ID(s): {document_ids}")
        for doc_id in document_ids:
            print(f"Document ID: {doc_id}, Document Name: {document_mapping[int(doc_id)]}")
    else:
        print(f"I'm sorry, I couldn't find the word '{word}' in any document.")

lookup_word('supply')

Congratulations! I could find the word 'supply' in the following document ID(s): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '12', '13', '14', '15', '16', '17', '18']
Document ID: 0, Document Name: A thousand ways to make money.txt
Document ID: 1, Document Name: Dumbells of Business by Louis Custer Martin Reed.txt
Document ID: 2, Document Name: Business Administration- Theory, Practice and Application.txt
Document ID: 3, Document Name: Cyclopedia of Commerce, Accountancy, Business Administration, v. 01 (of 10).txt
Document ID: 4, Document Name: Confessions of a Tradesman by Frank Thomas Bullen.txt
Document ID: 5, Document Name: Fundamentals of Prosperity- What They Are and Whence They Come by Roger Ward Babson.txt
Document ID: 6, Document Name: The Knack of Managing by Lewis K. Urquhart and Herbert Watson.txt
Document ID: 7, Document Name: Cyclopedia of Commerce, Accountancy, Business Administration, v. 04 (of 10).txt
Document ID: 8, Document Name: Creating Capital by Fred